In [1]:
pip install qiskit

  Using cached qiskit-1.2.4-cp38-abi3-win_amd64.whl.metadata (13 kB)
Using cached qiskit-1.2.4-cp38-abi3-win_amd64.whl (4.6 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\Sarthak\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\qiskit\\_accelerate.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
pip install qiskit_machine_learning


  Using cached qiskit-1.2.4-cp38-abi3-win_amd64.whl.metadata (13 kB)
  Using cached scikit_learn-1.5.2-cp313-cp313-win_amd64.whl.metadata (13 kB)
  Using cached fastdtw-0.3.4.tar.gz (133 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached setuptools-75.2.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached qiskit-1.2.4-cp38-abi3-win_amd64.whl (4.6 MB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --------- ------------------------------ 2.6/11.0 MB 11.9 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.0 MB 20.1 MB/s eta 0:

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\Sarthak\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\qiskit\\_accelerate.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
# Importing necessary libraries for classical anomaly detection models
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC  # Classical Support Vector Classifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from qiskit import Aer
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit_machine_learning.algorithms import QSVC
from qiskit.utils import QuantumInstance

# Step 1: Load the dataset
file_path = '/mnt/data/creditcard.csv'
df = pd.read_csv(file_path)

# Step 2: Data Cleaning and Preprocessing
# Dropping irrelevant features (like 'Time') if present
if 'Time' in df.columns:
    df.drop(columns=['Time'], inplace=True)

# Handling missing values if there are any
if df.isnull().sum().sum() > 0:
    df.fillna(df.mean(), inplace=True)

# Step 3: Normalization of features for consistency
scaler = StandardScaler()
features = df.drop(columns=['Class']).columns  # Assuming 'Class' is the target column
df[features] = scaler.fit_transform(df[features])

# Step 4: Feature Reduction using PCA
pca = PCA(n_components=10)  # Reducing to 10 principal components
principal_components = pca.fit_transform(df[features])
df_pca = pd.DataFrame(data=principal_components, columns=[f'PC{i+1}' for i in range(10)])
df_pca['Class'] = df['Class']  # Adding the target column back

# Step 5: Splitting the data into training and testing sets
X = df_pca.drop(columns=['Class']).to_numpy()
y = df_pca['Class'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 6: Classical Model - Support Vector Classifier (SVC)
print("Training the classical Support Vector Classifier...")
svc = SVC(kernel='rbf')  # Using Radial Basis Function kernel
svc.fit(X_train, y_train)
y_pred_classical = svc.predict(X_test)

# Evaluation of the Classical Model
print("\nClassical SVC Model Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred_classical))
print("Classification Report:\n", classification_report(y_test, y_pred_classical))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_classical))

# Step 7: Quantum Model - Quantum Support Vector Classifier (QSVC)
print("Preparing data for quantum computation...")

# Normalizing the dataset for amplitude encoding
norms = np.linalg.norm(X_train, axis=1)
X_train_normalized = X_train / norms[:, np.newaxis]
norms_test = np.linalg.norm(X_test, axis=1)
X_test_normalized = X_test / norms_test[:, np.newaxis]

# Quantum Feature Map for QSVC
feature_map = RawFeatureVector(feature_dimension=X_train_normalized.shape[1])
quantum_instance = QuantumInstance(Aer.get_backend('qasm_simulator'))

# Quantum SVC setup
qsvc = QSVC(quantum_instance=quantum_instance, feature_map=feature_map)
print("Training the Quantum Support Vector Classifier...")
qsvc.fit(X_train_normalized, y_train)
y_pred_quantum = qsvc.predict(X_test_normalized)

# Evaluation of the Quantum Model
print("\nQuantum SVC Model Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred_quantum))
print("Classification Report:\n", classification_report(y_test, y_pred_quantum))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_quantum))

# Step 8: Comparative Analysis
print("\nComparative Analysis of Classical vs Quantum Models:")
print(f"Classical Model Accuracy: {accuracy_score(y_test, y_pred_classical):.2f}")
print(f"Quantum Model Accuracy: {accuracy_score(y_test, y_pred_quantum):.2f}")

# Observations on potential improvements in performance
if accuracy_score(y_test, y_pred_quantum) > accuracy_score(y_test, y_pred_classical):
    print("Quantum model shows improved accuracy over the classical model.")
else:
    print("Classical model performs better or comparable to the quantum model in this scenario.")

# Step 9: Analysis of Speed and Scalability
import time

# Timing Classical Model Training
start_time_classical = time.time()
svc.fit(X_train, y_train)
classical_training_time = time.time() - start_time_classical
print(f"Classical Model Training Time: {classical_training_time:.2f} seconds")

# Timing Quantum Model Training
start_time_quantum = time.time()
qsvc.fit(X_train_normalized, y_train)
quantum_training_time = time.time() - start_time_quantum
print(f"Quantum Model Training Time: {quantum_training_time:.2f} seconds")

# Conclusion based on speed comparison
if quantum_training_time < classical_training_time:
    print("Quantum model shows potential speed advantage over the classical model.")
else:
    print("Classical model is faster in training compared to the quantum model in this setup.")

# Saving the processed data and code for submission
processed_data_path = '/mnt/data/processed_creditcard_data.csv'
df_pca.to_csv(processed_data_path, index=False)
print(f"\nProcessed dataset saved at: {processed_data_path}")


ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.

In [11]:
pip install qiskit qiskit-aer qiskit-machine-learning qiskit-ignis


  Using cached qiskit-1.2.4-cp38-abi3-win_amd64.whl.metadata (13 kB)
  Using cached qiskit-aer-0.15.1.tar.gz (6.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached qiskit_machine_learning-0.7.2-py3-none-any.whl.metadata (12 kB)
  Using cached qiskit_ignis-0.7.1-py3-none-any.whl.metadata (10 kB)
  Using cached qiskit_algorithms-0.3.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached retworkx-0.15.1-py3-none-any.whl.metadata (10 kB)
Using cached qiskit-1.2.4-cp38-abi3-win_amd64.whl (4.6 MB)
Using cached qiskit_machine_learning-0.7.2-py3-none-any.whl (97 kB)
Using cached qiskit_algorithms-0.3.0-py3-none-any.whl (308 kB)
Failed to build qiskit-aer
Note: you may need to restart the kernel to use u

  error: subprocess-exited-with-error
  
  × Building wheel for qiskit-aer (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [397 lines of output]
      
      
      --------------------------------------------------------------------------------
      -- Trying 'Ninja (Visual Studio 17 2022 x64 v144)' generator
      --------------------------------
      ---------------------------
      ----------------------
      -----------------
      ------------
      -------
      --
      Not searching for unused variables given on the command line.
      -- The C compiler identification is unknown
      CMake Error at CMakeLists.txt:3 (ENABLE_LANGUAGE):
        No CMAKE_C_COMPILER could be found.
      
        Tell CMake where to find the compiler by setting either the environment
        variable "CC" or the CMake cache entry CMAKE_C_COMPILER to the full path to
        the compiler, or to the compiler name if it is in the PATH.
      
      
      -- Configuring incomple